## 人工神经网络原理及实践

---

#### 介绍

神经网络中最核心的两个部分便是正向传播和反向传播。本实验将从神经网络的基础结构说起，分别对正向传播和反向传播进行阐述。然后，利用 NumPy 对其进行实现，完成一个完整的神经网络结构。最后，我们使用该神经网络，实现对数字图像进行识别。

#### 知识点

- 全连接
- 激活函数
- 独热编码
- 损失函数
- 梯度下降
- 反向传播

---

### 数据准备

本次实战我们将手写神经网络并对 scikit-learn 中的手写字符数据集 DIGITS 进行识别。

在学习神经网络之前，我们先来查看一下需要使用到的数据集。该数据集包含由 1797 张数字 0 到 9 的手写字符影像转换后的数字矩阵，目标值是 0-9。为了方便，这里直接使用 scikit-learn 提供的 `load_digits` 方法加载该数据集。

<i class="fa fa-arrow-circle-down" aria-hidden="true"> 教学代码：</i>

In [ ]:
from sklearn import datasets

digits = datasets.load_digits()
digits

<i class="fa fa-arrow-circle-down" aria-hidden="true"> 动手练习</i>｜如果你对课程所使用的实验楼 Notebook 在线环境并不熟悉，可以先学习 [<i class="fa fa-external-link-square" aria-hidden="true"> 使用指南课程</i>](https://www.shiyanlou.com/courses/1322)。

加载完成的 DIGITS 数据集中包含 3 个属性：

|  属性  |                  描述                  |
|:------:|:--------------------------------------:|
| `images` | 8x8 矩阵，记录每张手写字符图像对应的像素灰度值 |
|  `data`  | 将 images 对应的 8x8 矩阵转换为行向量 |
| `target` |     记录 1797 张影像各自代表的数字     |

我们可以根据灰度值矩阵，使用 Matplotlib 把字符对应的灰度图像和标签显示出来看看：

In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

image1 = digits.images[0]
print("标签为:", digits.target[0])
plt.imshow(image1, cmap=plt.cm.gray_r)

从图中可以看到，我们需要识别的图片是 $8 \times 8$ 的灰度图，它们的标签和图片内容一一对应。接下来，我们将在学习神经网络基本结构的同时，利用 Python 手写神经网络并完成对上述数据集的识别。

### 神经网络的全连接

其实神经网络的来源有多种说法，有些人喜欢将生物大脑的神经元网络结构看做是神经网络的灵感来源。大脑神经元的结构如下:

<img width='400px'  src="https://doc.shiyanlou.com/courses/uid1166617-20191111-1573466946585">
<div style="color: #888; font-size: 10px; text-align: right;"><a href="https://en.wikipedia.org/wiki/Neuron#/media/File:Neuron_Hand-tuned.svg/"><i class="fa fa-copyright" aria-hidden="true"> 来源</i></a></div>


我们主要关注细胞体、树突、轴突等结构。树突作为接收端，接收从其他神经元传导过来的神经冲动。而轴突将经过细胞体处理的神经冲动传递给其他神经元。从上图可以看出，一个神经元的树突可以有多个，但轴突却只有一个。其实这里的树突就可以看成一个神经元的输入，而轴突其实就是输出。

人工神经网络的结构如下：

<img width='400px'  src="https://doc.shiyanlou.com/courses/uid1166617-20191111-1573466852472">

你会发现这张图片和上面的大脑神经元的图片有很多相似之处，只不过神经冲动在这里变成了具体的数值。

图中的圆圈代表一个个神经元（ Neuron ），其中网络层 1（输入层） 中的每个神经元都与网络层 2 中的每个神经元相连。可以说 $a_1$，$a_2$，$a_3$ 就是一条数据的属性值（输入层的个数和数据的特征个数相同）。$h_1$，$h_2$ 可以看作为第二层网络层的输出。

同时我们注意到，神经元间的连接线上有权重 $w$ 。神经网络工作时，将前一层神经元的输出与权重 $w$ 相乘再加上一个偏移量 $bias$ 得到的结果，传递给下一层神经元。即有：

$$\mathrm { w }_{11} * \mathrm { a_l } + \mathrm { w }_{12}  * \mathrm { a } _{2 }+ \mathrm { w }_{ 13 }* \mathrm { a } _{3 }+ \mathrm { bias } _{1 }= \mathrm { b } _{1}  \tag 1 $$

$$\mathrm { w }_{21} * \mathrm { a_l } + \mathrm { w }_{22}  * \mathrm { a } _{2 }+ \mathrm { w }_{ 23 }* \mathrm { a } _{3 }+ \mathrm { bias } _{2 }= \mathrm { b } _{2} \tag 2$$

写成矩阵的形式：

$$\left[ \begin{array} { l l l } { w _ { 11 } } & { w _ { 12 } } & { w _ { 13 } } \\ { w _ { 21 } } & { w _ { 22 } } & { w _ { 23 } } \end{array} \right] \cdot \left[ \begin{array} { c } { a _ { 1 } } \\ { a _ { 2 } } \\ { a _ { 3 } } \end{array} \right] + \left[ \begin{array} { c } { b i a s _ { 1 } } \\ { b i a s _ { 2 } } \end{array} \right] = \left[ \begin{array} { l } { b _ { 1 } } \\ { b _ { 2 } } \end{array} \right] \tag 3$$

全连接的含义其实就是每层神经元都和下一层的所有神经元相连。

本质上讲，神经网络就是随便给定一组 $w$ 和 $bias$，再判断在该 $w$，$bias$ 条件下模型的好坏，再通过一定的算法对 $w$ 和 $bias$ 进行更新。如此循环，直到求出最佳的 $w$ 矩阵和 $bias$ 矩阵的值。求取这些参数的过程其实就是模型的训练（学习）过程。

我们把数据在网络层中从左到右计算的过程称之为「正向传播」，下面使用 Python 将全连接的正向传播过程进行实现。

In [ ]:
import numpy as np


class FullyConnect:
    # 面向对象中的构造函数，初始化该类的时候，会被调用
    # 传入参数 len_x 为输入数据的特征长度（也就是第一层的神经元个数）
    # len_y 为输出数据的个数（也就是第二层的神经元个数）
    def __init__(self, len_x, len_y):
        # 使用随机数初始化参数，请暂时忽略这里为什么多了 np.sqrt(l_x)
        # 从上面的 w 的矩阵形式可以发现， m 个神经元的网络层到n个神经元的网络层之间的 w 矩阵的大小为( n*m )
        self.weights = np.random.randn(len_y, len_x) / np.sqrt(len_x)
        self.bias = np.random.randn(len_y, 1)  # 使用随机数初始化参数，bias 的个数之后输出层的个数有关
        self.lr = 0  # 先将学习速率初始化为 0 ，最后统一设置学习速率

    # 全连接的正向传播过程，输入的便是训练数据
    def forward(self, x):
        self.x = x  # 把中间结果保存下来，以备反向传播时使用
        # 计算全连接层的输出，也就是上面矩阵乘法公式的代码表示
        self.y = np.array([np.dot(self.weights, xx) + self.bias for xx in x])
        return self.y  # 将这一层计算的结果向前传递

由于对于神经网络来说，一条样本只能占一行，因此这里我们还需要把大小 $8 \times 8$ 的图片转换成一个行向量传入神经网络中。幸运的是 DIGITS 数据集中的 `data` 属性已经为我们做好了这一点。我们查看一下前10张图片的行向量：

In [ ]:
digits.data[0:2].shape

接下来，我们把这个行向量传入全连接中，并且输出全连接层的预测结果。

In [ ]:
fully_connet = FullyConnect(64, 1)  # 传入网络层1，网络层2的长度
full_result = fully_connet.forward(digits.data[0:2])
full_result  # 这里只传入两条数据用于测试。得到一次正向传播后，两张图片的预测值

### 神经网络的激活函数

#### 激活函数概述

上面所说的全连接层的公式其实就是多元一次的乘法，也就是我们常说的线性运算。而神经元的功能远远不止这些，下面我们单独将一个神经元给提出来，看看里面的结构：

<img width='400px' src="https://doc.shiyanlou.com/courses/uid1166617-20191111-1573467073383">

从上图可以看到，神经元除了上面所说的线性转换之外，其实还包含了一个非线性转换:

$$h = g(z) \tag 4$$

而这个非线性转换函数 $g(z)$ 就叫做激活函数。

#### 非线性激活函数的优点

有多种理由要求我们必须向神经网络中引入非线性运算。在多层神经网络中，如果没有非线性运算部分，则多个线性运算可以被一个线性运算替换，那么多层次的网络结构就会失去意义（由于矩阵乘法的结合性）。

矩阵乘法的结合性： $b_1 = a_1 \cdot k_1 \cdot k_2$ 与 $b_1=a_1\cdot (k_1\cdot k_2)$ 等价。假设 $k_1 = 3,k_2 = 2$，则 $a_1$ 可以通过先乘以 $2$ ，再乘以$3$，这两次线性变换得到 $b_1$。$a_1$ 也可以通过直接乘以 $6$ ，这一次线性变换得到 $b_1$。

当然还有个比较重要的原因就是非线性的分类效果会比线性的好。为了更好的展示，我们以二维数据为例。假设数据有两个特征 $a_1$，$a_2$ 。且他们最后被分为了两类。如下图所示：

<img width='600px' src="https://doc.shiyanlou.com/courses/uid1166617-20191113-1573608633813">

其中红色和绿色是实际的两种类别，而蓝色的线则是最终训练好的分类模型。由图可以看出，针对于该数据集，线性分类器无法准确的进行分类。 而非线性部分的引入，在一定程度上可以使原本的直线变成曲线，进而得到更好的分类效果，这就是激活函数引入的原因。

#### 激活函数的实现

实际运用当中，有多种激活函数可以选择，你甚至可以自己定义一个属于自己的激活函数。这里我们介绍最经典的一种激活函数：Sigmoid 激活函数。它的数学形式为：

$$h = g ( z ) = \frac { 1 } { 1 + e ^ { - z } } \tag 5$$

将全连接输出的数据 $z$ ，放入激活函数中，最终得到该神经元的输出。现在我们利用 Python 自己定义一个 Sigmoid 函数，并且完成他的正向传播过程。最后将的图像画出来看看。

In [ ]:
class Sigmoid:
    def __init__(self):  # 无参数，不需初始化
        pass
    # 这里输入的变量的 x，其实就是上面公式的 z

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    # 完成正向传播，将输入的 z ，放入 Sigmoid 函数中，最终得到结果 h，并返回
    def forward(self, x):
        self.x = x
        self.y = self.sigmoid(x)
        return self.y

接下来，我们利用所写的 Sigmoid 函数，将其图像画出：

In [ ]:
sig = Sigmoid()
print(sig.sigmoid(0))  # 当 z=0 时，正确输出应该是 0.5
%matplotlib inline
z = np.linspace(-6, 6, 1000)  # 生成等间距 z 值方便绘图
plt.plot(z, sig.sigmoid(z))
plt.grid(1)  # 生成网格，方便观察
plt.xlabel("z")
plt.ylabel("g(z)")

### 神经网络的损失函数

上面我们讲了神经网络的全连接，以及激活函数的正向传播过程。很多神经网络其实就是让数据不断的通过全连接层和激活函数层，最终得到预测结果。那么问题来了，得到预测结果后，如何说明当前状态下的模型是优还是劣呢？神经网络是否还需要继续训练下去呢？为此，我们引入了损失函数的概念。

#### 损失函数

损失函数，顾名思义，就是模型预测出来的标签与真实标签的差异。而定义这种差异的函数，就被称为损失函数。深度学习的训练过程其实就是求解损失函数最小值的过程。比如计算真实值和预测值之间的绝对误差，当得到的值比较大时，就说明该神经网络的输出与预期的正确输出偏差较大。反之，如果得到的值很小甚至等于 0 ，就说明我们的模型工作的不错，能够正确的预测输出值。

实际上，现在已经有很多种损失函数供我们选择，这里我们就来介绍一种最经典的损失函数：二次损失函数（Quadratic Loss Function）。

#### 二次损失函数

就像它的名字所暗示的那样，Quadratic Loss Function 通过计算 Predict 和 Real 之间差值的平方来衡量神经网络的优劣。具体形式如下：

$$J ( \Theta ) = \frac { 1 } { 2 m } \sum _ { i = 1 } ^ { m } \left( h \left( \Theta , X _ { i } \right) - Y _ { i } \right) ^ { 2 } \tag 6 $$

其中的希腊字母 $\Theta$ 表示模型的参数，也就是上面的 $W$ 和 $bias$ 。$X_i$ 表示第 $i$ 个样本数据，$Y_i$ 代表输入 $X_i$ 所对应的，正确的输出值。$h ( \Theta , X_i )$ 表示将 $X_i$ 输入神经网络，然后通过正向传播后得到的预测值。

函数的意义：求每条数据的损失，然后加起来再取平均得到整个训练数据集的平均损失过程。当然至于这里为什么最后需要除以一个 $2$，容我们放到后面来讲。不过这里除以 $2$ ，对整个神经网络是没有影响的。因为当 $1/2$ 损失函数最小时，损失函数自然也会最小。

为什么损失函数是关于 $\Theta$ 的函数？因为模型的训练集 $X$ 是给定的，而真正的变量其实是 $\Theta$ (也就是每一层的 $W$ 和 $bias$ )，当确定好这些参数后，整个神经网络就训练成功了。

#### 独热编码

生活中标签 $Y_i$ 的形式各种各样，有可能是预测天气的阴天，晴天，雨天等标签，也可能是预测字母的 $a,b,c...z$ 等。而如何将这些标签转换成计算机能够识别的标签呢？有很多种方式，比如十进制。但是如果使用十进制来表示这些离散标签的话，会有一个缺点。假设我把 $0$ 当做晴天，$1$ 当做雨天，$2$ 当做阴天。那么在计算损失时，（晴天，阴天）的损失和（晴天，雨天）的损失会不同。可他们都是把标签预测错了，没有理由让他们的损失不同。因此便提出了独热编码的概念。

独热编码：数字的每一位只有 0 和 1 的取值，且每一个都代表一个标签，如果这位取1，其他位则必须为0。如下图所示：

<img width='300px' src="https://doc.shiyanlou.com/courses/uid1166617-20191105-1572934493224">

当第 0 位为 1 ，其他位为 0 的时候，则表示晴天。当第 1 位为 1，其他位为 0 的时候，则表示雨天，其他的标签同理。这里把他们看做向量坐标，则晴天与阴天的距离和晴天与雪天的距离都为 1。这样计算出来的损失也就相等了。

#### 损失函数的编写

这里我们利用 Python 对二次损失函数层进行编写，代码如下：

In [ ]:
class QuadraticLoss:
    def __init__(self):
        pass
    # 传入的参数，第一个参数为预测出来的标签值，第二个参数为实际标签值
    def forward(self, x, label):
        # 将真实 label 转换成独热编码
        self.x = x
        # 由于我们的label本身只包含一个数字，我们需要将其转换成和模型输出值尺寸相匹配的向量形式
        self.label = np.zeros_like(x)
        for a, b in zip(self.label, label):
            a[b] = 1.0  # 只有正确标签所代表的位置概率为1，其他为 0
        # 计算损失
        self.loss = np.sum(np.square(x - self.label)) / \
            self.x.shape[0] / 2  # 求平均后再除以 2 是为了表示方便
        return self.loss

接下来，我们初始化上面所说的晴天，雨天，阴天，雪天等四种天气。然后，利用所写损失函数，观察（阴天，晴天）的损失与（雪天，晴天）的损失是否相同。

In [ ]:
# 测试
loss = QuadraticLoss()
# 假设神经网络算出样本的预测值为0，即为晴天
pred = np.zeros((1, 4))
pred[0] = 1
print("实际为阴天和预测值为晴天的平均损失是：", loss.forward(pred, [1]))
print("实际为雪天和预测值为晴天的平均损失是：", loss.forward(pred, [2]))

从结果可以看出，通过独热编码后的（阴天，晴天）的损失与（雪天，晴天）的损失相同。

### 准确率函数

之前我们提到了损失函数是用来衡量神经网络是否训练的一个标准，不同的损失函数可能计算出来的损失值的量级可能不同。那么我又该如何来衡量不同的神经网络结构的优劣呢？最简单的方法就是看这个神经网络模型他到底预测正确多少数据，预测错误多少数据，这就是准确率，公式如下：

 $$ Accuracy = \frac{{correct}}{{sum}} \tag{7}$$

其中 Correct 表示预测正确的样本数，Sum 表示预测的总样本数。接下来，我们对准确率进行编写：

In [ ]:
class Accuracy:
    def __init__(self):
        pass

    def forward(self, x, label):  # 只需forward
        self.accuracy = np.sum(
            [np.argmax(xx) == ll for xx, ll in zip(x, label)])  # 对预测正确的实例数求和
        self.accuracy = 1.0 * self.accuracy / x.shape[0]  # 也就是计算正确率 ,公式 7 的实现
        return self.accuracy

### 神经网络正向传播

我们已经写好了所有网络层的正向传播，接下来我们要使用这些网络层构建出一个完整的神经网络的正向传播。并传入需要预测的数据集，进行一次正向传播，查看输出结果。

In [ ]:
# 图片大小为 8*8
# 则此时一张图片就是一条数据，每张图片对呀一个 label（0-9范围内）
x = digits.data
labels = digits.target

# 开始搭建神经网络
inner_layers = []
inner_layers.append(FullyConnect(8 * 8, 10))
inner_layers.append(Sigmoid())
# 神经网络搭建完成

losslayer = QuadraticLoss()  # 计算损失
accuracy = Accuracy()  # 计算准确率

# 开始将数据送入神经网络进行正向传播
for layer in inner_layers:  # 前向计算
    x = layer.forward(x)

loss = losslayer.forward(x, labels)  # 调用损失层forward函数计算损失函数值
accu = accuracy.forward(x, labels)
print('loss:', loss, 'accuracy:', accu)

根据答案可以看出，一次的正向传播之后，模型的损失很大，正确率接近为 0 。那么有没有什么办法可以减少损失进而提高正确率呢？接下来我们就会学习一种求解损失最小值的方法：梯度下降算法。

### 梯度下降算法

我们已经学习完了整个神经网络的正向传播过程。正如上文所述，神经网络的学习过程其实就是求解 $W$，$bias$ 在最佳状态下的值。而最佳状态的衡量标准便是损失函数最小。那么怎样求解损失函数最小情况下的参数值呢？这就需要介绍到一种求解求函数最小值的算法：梯度下降算法。

梯度下降算法是一种求解函数极小值的方法，他的中心思想其实很简单，如下图所示，假设你身处峨眉山之中，现在你想以最快的速度下山，那么你应该朝着哪个方向走呢？毋庸置疑，朝着下方，且最陡的方向（如果是垂直的更好，可以马上跳下去，最快的到达山脚）。当然上面只是个比喻，一切还是安全第一。这里我们所说的最陡的方向，用数学的语言表达便是梯度的相反方向（因为梯度表示函数上升最快的方向）。梯度本身是一个向量，由函数对每个自变量分别求偏导得到。

<img width='500px' src="https://doc.shiyanlou.com/courses/uid1166617-20191113-1573623914486">

#### 公式推导

现在我们有了下山的方向，那么假设我们现在所在位置为点 $\Theta$ 。当我朝着梯度相反的方向走了一步后，就到达新的一个点，然后再在这个点上求梯度，再朝着新的梯度相反的方向往下走，这样就能很快的到达山脚，这就是梯度下降的原理。通过公式来表达，就是：

$$\Theta = \Theta - \alpha \frac { d J } { d \Theta } \tag 8$$

注意，这里多了一个希腊字母 $\alpha$, 因为梯度只为我们指明了前进（更新）的方向，而我们具体朝着这个方向「走多远」则由 $\alpha$ 控制。因此 $\alpha$ 还有一个朴素的名字叫做步长（一步走多远的意思），又叫学习率。

#### 步长的设置

在深度学习中步长的设计很有讲究。虽然 $\alpha$ 越大，参数 $\Theta$ 更新的幅度就会越大，我们就可能越可能更快的到达最低点。但是 $\alpha$ 不能设置的太大，否则有可能因为步子太大，而直接越过最低点，甚至导致损失函数值不降反升。如下图所示：

<img width='600px' src="https://doc.shiyanlou.com/courses/uid1166617-20191111-1573468513574">

因此，设置合理的学习速率 $\alpha$ 非常重要。实际当中的做法一般是先设置 $\alpha$ 为一个较小的值，比如 $0.001$ 。观察每一次参数 $\theta$ 更新后损失函数 $J$ 的变化。如果 $J$ 变大了，就将 $\alpha$ 的值除以 $10$，变成 $0.0001$ ，直到损失函数值开始变小。

如果 $J$ 变小了，则可以将 $\alpha$ 再乘以 $10$ ，使得学习的速率更快。最终使 $\alpha$ 停留在一个能够使 $J$ 变小，又不会因为值过小导致学得太慢的「临界值」。

### 反向传播

下图黑色的箭头展示了，数据正向传导情况，而红色的则反映了反向传播情况。通过正向传播得到的预测值，算出当前 $W$ 和 $b$ 下的损失，然后再通过一层一层的偏导最终得到参数 $W$ 和 $b$ 的变化值 $dw$，$db$ ,然后根据梯度下降的思想，更新参数，再正向传播，如此循环。当损失降到最小或者不变时则停止。

<img width='700px' src="https://doc.shiyanlou.com/courses/uid1166617-20191105-1572946801966">

下面我们将按照反向传播的顺序，利用梯度下降的原理，分别对每种类别的网络层进行编写。

#### 损失函数层的反向传播

损失函数的公式如下：

$$J ( \Theta ) = \frac { 1 } { 2 m } \sum _ { i = 1 } ^ { m } \left( h \left( \Theta , X _ { i } \right) - Y _ { i } \right) ^ { 2 } \tag 9$$

该公式的具体含义由于上文已有，这里就不做阐述了。这里需要阐述一下的是，为什么得到样本的平均损失之后需要除以 $2$ 。这是为了在求梯度（偏导）时，抵消从指数下来的那一个 $2$ 。因此将损失函数层的反向传播添加到原来所写的损失函数中，得到新的损失函数层，如下：

In [ ]:
class QuadraticLoss:
    def __init__(self):
        pass
    # 正向传播和上文一样，具体注释参照上文
    def forward(self, x, label):
        self.x = x
        self.label = np.zeros_like(x)
        for a, b in zip(self.label, label):
            a[b] = 1.0
        # 对公式 8 实现
        self.loss = np.sum(np.square(x - self.label)) / \
        self.x.shape[0] / 2  # 求平均后再除以2是为了表示方便
        return self.loss

    # 定义反向传播
    def backward(self):
        # 这里的dx，就是我们求得函数关于x偏导数，也就是梯度，将它保存起来，后面更新的时候会用到
        self.dx = (self.x - self.label) / self.x.shape[0]  # 2被抵消掉了
        return self.dx

#### 激活函数的反向传播

这里我们选择的激活函数是 Sigmoid 函数，首先需要对这个函数进行求导。Sigmoid 函数的求导过程这里就不阐述了，如果读者有兴趣可以自行推导。Sigmoid 函数的导数如下：

$$sigmoid(x)'=sigmoid(x)*(1-sigmoid(x)) \tag {10}$$

现在我们就可以通过 Sigmoid 函数的导数，对激活函数层的反向传播进行编写，激活函数的全部代码如下：

In [ ]:
class Sigmoid:
    def __init__(self):  # 无参数，不需初始化
        pass
    # 即公式 5
    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))

    def forward(self, x):
        self.x = x
        self.y = self.sigmoid(x)
        return self.y
    # 即公式 9
    def backward(self, d):
        sig = self.sigmoid(self.x)
        self.dx = d * sig * (1 - sig)
        return self.dx  # 反向传递梯度

#### 全连接层的反向传播

现在我们已经走到了最后一步，那就是全连接层的反向传播过程。这个过程也是最重要的过程，他将接收激活函数层传递过来的，处理后的损失误差。而这一层也将通过损失误差，计算相应的参数 $w$,$b$ 的梯度 $dw$，$db$ 。而其中参数 $w$，$b$ 与最后的结果产生联系的过程如下所示：

$$\mathrm { z } = \mathrm { w }_{11} * \mathrm { a_l } + \mathrm { w }_{12} * \mathrm { a }_2 + \mathrm { b }_1 \tag{11}$$

$$\mathrm { h } = \operatorname { sigmoid } ( \mathrm { z } )\tag {12}$$

$$\mathrm { h }_1 = \operatorname { sigmoid } \left( \mathrm { w }_{11} * \mathrm { a }_1 + \mathrm { w }_{12} * { \mathrm { a }_{2} } + \mathrm { b }_1 \right)\tag {13}$$

也就是说，我们求的其实是 $h_1$ 关于 $w$ 和 $b$ 的偏导。最终我们可以得到

$$dw = dz \cdot X^T \tag{14}$$

$$db =dz \tag{15}$$

因此，我们开始改写全连接层，并且最后利用梯度下降对参数进行更新。

In [ ]:
class FullyConnect:
    def __init__(self, l_x, l_y):  # 两个参数分别为输入层的长度和输出层的长度
        # 使用随机数初始化参数，请暂时忽略这里为什么多了np.sqrt(l_x)
        self.weights = np.random.randn(l_y, l_x) / np.sqrt(l_x)
        self.bias = np.random.randn(l_y, 1)  # 使用随机数初始化参数
        self.lr = 0  # 先将学习速率初始化为0，最后统一设置学习速率

    def forward(self, x):
        self.x = x  # 把中间结果保存下来，以备反向传播时使用
        self.y = np.array([np.dot(self.weights, xx) +
                           self.bias for xx in x])  # 计算全连接层的输出
        return self.y  # 将这一层计算的结果向前传递

    def backward(self, d):
        # 根据链式法则，将反向传递回来的导数值乘以x，得到对参数的梯度
        ddw = [np.dot(dd, xx.T) for dd, xx in zip(d, self.x)]
        # 每一条数据都能求出一个ddw，然后对他们取一个平均，得到平均的梯度变化
        self.dw = np.sum(ddw, axis=0) / self.x.shape[0]
        self.db = np.sum(d, axis=0) / self.x.shape[0]
        self.dx = np.array([np.dot(self.weights.T, dd) for dd in d])

        # 利用梯度下降的思想，更新参数。这里的lr就是步长的意思
        self.weights -= self.lr * self.dw
        self.bias -= self.lr * self.db
        return self.dx  # 反向传播梯度

我们已经了解完了反向传播的全部过程。接下来，我们就会用到上文所写的各大网络层函数，搭建一个完整的神经网络，对图像进行识别。

### 手写数字识别

这里我们将数据的前 1500 条作为训练数据，后面的作为测试数据。得到如下数据集：

In [ ]:
train_data,train_target = digits.data[:1500],digits.target[:1500]
test_data,test_target = digits.data[1500:-1],digits.target[1500:-1]
train_data.shape,train_target.shape,test_data.shape,test_target.shape

接下来，我们利用上面所写的网络层，搭建一个用于数字识别的网络结构。该网络结构由（全连接层，激活函数层，全连接层，激活函数）组成。具体代码如下：

In [ ]:
inner_layers = []
inner_layers.append(FullyConnect(64, 60)) # 因为每条数据的长度为 8*8=64，因此这里第一个全连接层，接收长度为64
inner_layers.append(Sigmoid())
inner_layers.append(FullyConnect(60, 10))
inner_layers.append(Sigmoid())
inner_layers

接下来，初始化损失函数，准确率函数，学习率以及迭代次数。

In [ ]:
losslayer = QuadraticLoss()
accuracy = Accuracy()
for layer in inner_layers:
    layer.lr = 1000     #所有中间层设置学习速率
epochs = 150  # 对训练数据遍历的次数，也就是学习时间。
#在开始的时候，准确率会随之学习时间的增加而提高。
#当模型学习完训练数据中的所有信息后，准确率就会趋于稳定
losslayer,accuracy,epochs

最后，对模型进行训练。且每训练 2 次，则输出一次测试结果。

In [ ]:
for i in range(epochs):
   
    losssum = 0
    iters = 0
    x = train_data
    label = train_target
    x = x.reshape(-1,64,1)
    for layer in inner_layers:  # 前向计算
        x = layer.forward(x)
    loss = losslayer.forward(x, label)  # 调用损失层forward函数计算损失函数值
    losssum += loss
    iters += 1
    d = losslayer.backward()  # 调用损失层backward函数层计算将要反向传播的梯度

    for layer in inner_layers[::-1]:  # 反向传播
        d = layer.backward(d)

    if i%10==0: 
        x = test_data
        label = test_target
        x = x.reshape(-1,64,1)
        for layer in inner_layers:
            x = layer.forward(x)
            
        accu = accuracy.forward(x, label)  # 调用准确率层forward()函数求出准确率
        print('epochs:{},loss:{},test_accuracy:{}'.format(i,losssum / iters,accu))

你可以通过设置不同的迭代次数以及学习率观察学习效果，准确率最终能够达到 85% 以上。

### 实验总结

我们从神经网络的原理出发，了解了神经网络的正向传播原理和反向传播原理。并在学习原理的过程中，利用 Python 搭建起了一个简单的神经网络模型。而很多解决实际问题的神经网络其实非常复杂。因此，为了提高效率，我们常常会采用 TensorFlow，PyTorch 等深度学习框架搭建神经网络。 

<hr><div style="color: #999; font-size: 12px;"><i class="fa fa-copyright" aria-hidden="true"> 本课程内容版权归实验楼所有，禁止转载、下载及非法传播。</i></div>